In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-1m-dataset/users.dat
/kaggle/input/movielens-1m-dataset/ratings.dat
/kaggle/input/movielens-1m-dataset/README
/kaggle/input/movielens-1m-dataset/movies.dat
/kaggle/input/movielens-100k-dataset/ml-100k/u.occupation
/kaggle/input/movielens-100k-dataset/ml-100k/u1.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.info
/kaggle/input/movielens-100k-dataset/ml-100k/u4.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.item
/kaggle/input/movielens-100k-dataset/ml-100k/README
/kaggle/input/movielens-100k-dataset/ml-100k/u1.test
/kaggle/input/movielens-100k-dataset/ml-100k/ua.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.data
/kaggle/input/movielens-100k-dataset/ml-100k/u5.test
/kaggle/input/movielens-100k-dataset/ml-100k/mku.sh
/kaggle/input/movielens-100k-dataset/ml-100k/u5.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.user
/kaggle/input/movielens-100k-dataset/ml-100k/ub.base
/kaggle/input/movielens-100k-dataset/ml-100k/u4.base
/kaggle/input/movielens-10

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

movies = pd.read_csv('/kaggle/input/movielens-1m-dataset/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/kaggle/input/movielens-1m-dataset/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/kaggle/input/movielens-1m-dataset/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
training_set = pd.read_csv('/kaggle/input/movielens-100k-dataset/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
testing_set = pd.read_csv('/kaggle/input/movielens-100k-dataset/ml-100k/u1.test', delimiter = '\t')
testing_set = np.array(testing_set, dtype = 'int')
no_users = int(max(max(training_set[:,0]), max(testing_set[:,0])))
no_movies = int(max(max(training_set[:,1]), max(testing_set[:,1])))

def convert_data(data):
    new_data = []
    for id_users in range(1, no_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(no_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert_data(training_set)
testing_set = convert_data(testing_set)

training_set = torch.FloatTensor(training_set)
testing_set = torch.FloatTensor(testing_set)

training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
testing_set[testing_set == 0] = -1
testing_set[testing_set == 1] = 0
testing_set[testing_set == 2] = 0
testing_set[testing_set >= 3] = 1

class RBM:
    def __init__(self, nv, nh):
        self.W = torch.randn(nh,nv)
        self.b = torch.randn(1, nv)
        self.a = torch.randn(1, nh)
        
    def sample_h(self, v):
        vw = torch.mm(v, self.W.T)
        activation = vw + self.a.expand_as(vw)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, h):
        hw = torch.mm(h, self.W)
        activation = hw + self.b.expand_as(hw)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, p_h0_given_v0, p_hk_given_vk):
        # Gibbs Sampling
        self.W += (torch.mm(v0.T, p_h0_given_v0)-torch.mm(vk.T, p_hk_given_vk)).T
        self.b += torch.sum((v0-vk), 0)
        self.a += torch.sum((p_h0_given_v0-p_hk_given_vk), 0)

nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

no_epochs = 10

for epoch in range(1, no_epochs + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, no_users-batch_size, batch_size):
        v0 = training_set[id_user:id_user+batch_size]
        vk = v0.clone()    
        p_h0_given_v0,_ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        p_hk_given_vk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, p_h0_given_v0, p_hk_given_vk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

test_loss = 0
s = 0.

for id_user in range(no_users):
    v = training_set[id_user:id_user+1]
    vt = testing_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

epoch: 1 loss: tensor(0.3588)
epoch: 2 loss: tensor(0.2525)
epoch: 3 loss: tensor(0.2465)
epoch: 4 loss: tensor(0.2537)
epoch: 5 loss: tensor(0.2448)
epoch: 6 loss: tensor(0.2381)
epoch: 7 loss: tensor(0.2431)
epoch: 8 loss: tensor(0.2470)
epoch: 9 loss: tensor(0.2489)
epoch: 10 loss: tensor(0.2454)
test loss: tensor(0.2353)
